Appendix A: Code

In [2]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

import cv2
from tqdm import tqdm_notebook, tnrange
from glob import glob
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from sklearn.metrics import *


import tensorflow as tf
from skimage.color import rgb2gray
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model, Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint,  ReduceLROnPlateau, LearningRateScheduler

from tensorflow.keras.preprocessing.image import ImageDataGenerator

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.applications import DenseNet121

In [3]:
train_files = []
mask_files = glob('../input/lgg-mri-segmentation/kaggle_3m/*/*_mask*')

for i in mask_files:
    train_files.append(i.replace('_mask',''))

In [4]:
def label(mask):
    value = np.max(cv2.imread(mask))
    return '1' if value > 0 else '0'

df = pd.DataFrame({"image": train_files,
                   "mask": mask_files,
                  "label":[label(x) for x in mask_files]})

df_train, df_test = train_test_split(df,test_size = 0.25, stratify=df['label'])
df_train, df_val = train_test_split(df_train,test_size = 0.4, stratify=df_train['label'])
print(df_train.values.shape)
print(df_val.values.shape)
print(df_test.values.shape)

In [5]:
def img_load(preprocess):
    train_generator_args = dict(rescale=1. / 255, 
                            rotation_range=0.2,
                            width_shift_range=0.05,
                            height_shift_range=0.05,
                            shear_range=0.05,
                            zoom_range=0.05,
                            horizontal_flip=True,
                            fill_mode='nearest',
                           preprocessing_function = preprocess)
    train_datagen = ImageDataGenerator(**train_generator_args)
    img_gen = ImageDataGenerator(rescale=1. / 255,  preprocessing_function = preprocess)
    
    train_gen = train_datagen.flow_from_dataframe(df_train,
                                                x_col = "image",
                                                y_col = 'label',
                                                class_mode = 'binary',
                                                 target_size = (256,256))
    
    val_gen = img_gen.flow_from_dataframe(
                                        df_val,
                                        x_col = "image",
                                        y_col = 'label',
                                        class_mode = 'binary',
                                        shuffle=False,
                                                 target_size = (256,256))

    test_gen = img_gen.flow_from_dataframe(
                                        df_test,
                                        x_col = "image",
                                        y_col = 'label',
                                        class_mode = 'binary',
                                        shuffle=False,
                                                 target_size = (256,256))
    return train_gen, val_gen, test_gen

In [6]:
def callbacklist(modelname='model'):
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', mode='min', verbose=1, patience=10,min_delta=0.0001, factor=0.2)
    checkpointer = ModelCheckpoint(modelname +'.hdf5', verbose=1, save_best_only=True)
    earlystopping = EarlyStopping(monitor='val_loss', mode='min', verbose=1,patience=20)
    callback_list = [reduce_lr, checkpointer, earlystopping]
    return callback_list

In [7]:
def loss_acc_plot(modelhistory):
    scores = pd.DataFrame(modelhistory.history)
    scores[['loss', 'val_loss']].plot();
    scores[['acc', 'val_acc']].plot();
def test_pred(model):
    y_pred = np.round(model.predict(test_gen))
    y_test = test_gen.classes
    print(classification_report(y_test, y_pred))
    return

# ResNet50

In [23]:
resnet_preprocess = tf.keras.applications.resnet50.preprocess_input

train_gen, val_gen, test_gen = img_load(resnet_preprocess)

In [25]:
model = Sequential()
model.add(ResNet50(include_top = False, weights = 'imagenet'))
model.add(GlobalAveragePooling2D())
model.add(Dropout(0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.layers[0].trainable = False 

model.summary()

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

callback_list = callbacklist('resnet50')

history = model.fit(train_gen,
                    epochs=30, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

In [26]:
model.layers[0].trainable = True

model.summary()

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_gen,
                    epochs=20, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

# VGG19

In [ ]:
from tensorflow.keras.applications import VGG19

resnet_preprocess = tf.keras.applications.vgg19.preprocess_input

train_gen, val_gen, test_gen = img_load(resnet_preprocess)



model = Sequential()
model.add(VGG19(include_top = False, weights = 'imagenet'))
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation = 'sigmoid'))
model.layers[0].trainable = False 

model.summary()

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

callback_list = callbacklist('vgg19')

history = model.fit(train_gen,
                    epochs=30, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

In [ ]:
model.layers[0].trainable = True

model.summary()

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_gen,
                    epochs=20, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

# InceptionV3

In [29]:
from tensorflow.keras.applications import InceptionV3

inception_preprocess = tf.keras.applications.inception_v3.preprocess_input

train_gen, val_gen, test_gen = img_load(inception_preprocess)

model = Sequential()
model.add(InceptionV3(include_top = False, weights = 'imagenet'))
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation = 'sigmoid'))
model.layers[0].trainable = False 

model.summary()

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

callback_list = callbacklist('inceptionv3')

history = model.fit(train_gen,
                    epochs=30, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

In [30]:
model.layers[0].trainable = True

model.summary()

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_gen,
                    epochs=20, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

# DenseNet121

In [31]:
dense_preprocess = tf.keras.applications.densenet.preprocess_input 

train_gen, val_gen, test_gen = img_load(dense_preprocess)

model = Sequential()
model.add(DenseNet121(include_top = False, weights = 'imagenet'))
model.add(GlobalAveragePooling2D())
model.add(Dense(1, activation = 'sigmoid'))
model.layers[0].trainable = False 

model.summary()

model.compile(optimizer=Adam(lr=0.001), loss='binary_crossentropy', metrics=['acc'])

callback_list = callbacklist('DenseNet121')

history = model.fit(train_gen,
                    epochs=30, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)

In [32]:
model.layers[0].trainable = True

model.summary()

model.compile(optimizer=Adam(lr=0.00001), loss='binary_crossentropy', metrics=['acc'])

history = model.fit(train_gen,
                    epochs=20, 
                    callbacks=callback_list,
                    validation_data = val_gen)

loss_acc_plot(history)
test_pred(model)